# Install libraries 


In [1]:
import sys, os, json
import mne, sklearn, wandb
import numpy as np
import pandas as pd

from scipy.interpolate import interp1d
from nilearn import datasets, image, masking, plotting
from nilearn.input_data import NiftiLabelsMasker


# animation part
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt
# from celluloid import Camera   # it is convinient method to animate
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation


## torch libraries 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader, Subset

from pytorch_model_summary import summary


/home/user/.local/lib/python3.9/site-packages/nilearn/datasets/__init__.py:93: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
%load_ext autoreload
%autoreload 2
sys.path.insert(1, os.path.realpath(os.path.pardir))

from utils import get_datasets
from utils import preproc
from utils import torch_dataset
from utils import train_utils
from utils import inference
from utils.models_arch import autoencoder_v3
from utils.models_arch import transformer_models 

# Set all hyperparameters
- Cuda and GPU.
- Parameters of dataset. 
- random seed( if necessary). 


In [3]:
print(torch.cuda.is_available(), torch.cuda.device_count())
torch.cuda.set_device(1)

True 4


In [4]:
config = dict(  
                dataset_name = 'CWL', # NODDI
                fps = 250,
                new_fps=250, 
                n_channels = 30, # 64 
                n_roi = 21,
                test_sec = 60, # in seconds.
                freqs = np.linspace(2, 100, 16), 
                
                
                bold_delay = 0,
                to_many = False,
                random_subsample = True,
                sample_per_epoch = 2048, 
                WINDOW_SIZE = 4096, # 16 sec.
                    
                optimizer='adam',
                lr=0.001,
                weight_decay=0.00001, 
                batch_size=16, 

                loss_function = 'mse', 
                model_type = 'strided medium vanilla transformer'
                )




n_time_points = 32 
config_feature_extractor = dict(stride = config['WINDOW_SIZE']//n_time_points)
# config_feature_extractor = dict(n_inp_features=n_features,
#                                  channels = [32, 32, 32, 32, 32], 
#                                  kernel_sizes=[9, 5, 3, 3, 3],
#                                  strides=[4, 4, 2, 2, 2], 
#                                  dilations = [1, 1, 1, 1, 1])


config_vanilla_transformer = dict(n_electrodes = config['n_channels'],
                                 sequence_length = n_time_points,
                                 embed_dim=len(config['freqs']),
                                 n_roi=config['n_roi'],
                                 num_heads=8,
                                 mlp_ratio=4,
                                 num_layers=4,
                                 attn_dropout=0.1,                
                                 mlp_dropout=0.5)

config = {**config_vanilla_transformer,**config_feature_extractor,  **config}

params_train = {'batch_size': config['batch_size'],
                'shuffle': True,
                'num_workers': 0}

params_val = {'batch_size': config['batch_size'],
              'shuffle': False}

In [5]:
# import random

# torch.manual_seed(0)
# random.seed(0)  # python operation seed
# np.random.seed(0)

# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True


# Upload preprocessed dataset from np files. 
It should accelerate speed of experiments.

In [6]:
with open("labels_roi.json", 'r') as f:
    labels_roi = json.load(f)


if config['dataset_name']=='CWL':
    dataset_path = '../data/dataset_cwl_250_hz.npz'
    
elif config['dataset_name']=='NODDI':
    dataset_path = '../data/dataset_NODDI_250_hz.npz'
else:
    print('no such dataset')


# download data
data = np.load(dataset_path)
train_dataset_prep = (np.clip(data['x_train'], 0, np.max(data['x_train'])), data['y_train'])
test_dataset_prep = (np.clip(data['x_test'], 0, np.max(data['x_test'])), data['y_test'])





# apply time dealy corrected
train_dataset_prep = preproc.bold_time_delay_align(train_dataset_prep, 
                                                   config['new_fps'],
                                                   config['bold_delay'])
test_dataset_prep = preproc.bold_time_delay_align(test_dataset_prep, 
                                                  config['new_fps'],
                                                  config['bold_delay'])


print('Size of train dataset:', train_dataset_prep[0].shape, train_dataset_prep[1].shape)
print('Size of test dataset:', test_dataset_prep[0].shape, test_dataset_prep[1].shape)

# torch dataset creation 
# torch dataset creation 
torch_dataset_train = torch_dataset.CreateDataset_eeg_fmri(train_dataset_prep, 
                                                            random_sample=config['random_subsample'], 
                                                            sample_per_epoch=config['sample_per_epoch'], 
                                                            to_many=config['to_many'], 
                                                            window_size = config['WINDOW_SIZE'])

torch_dataset_test = torch_dataset.CreateDataset_eeg_fmri(test_dataset_prep, 
                                                            random_sample=False, 
                                                            sample_per_epoch=None, 
                                                            to_many=config['to_many'], 
                                                            window_size = config['WINDOW_SIZE'])


# init dataloaders for training
train_loader = torch.utils.data.DataLoader(torch_dataset_train, **params_train)
val_loader = torch.utils.data.DataLoader(torch_dataset_test, **params_val)

Size of train dataset: (30, 16, 54225) (21, 54225)
Size of test dataset: (30, 16, 15000) (21, 15000)


# Init Model, Loss, optimizers

In [7]:
# x = torch.zeros(32, config['n_channels'], len(config['freqs']),config['WINDOW_SIZE'])
# print(x.shape), model(x).shape

In [8]:

feature_extraction_model = transformer_models.get_strided_func(**config_feature_extractor)
# conv_wav2net = transformer_models.wav2vec_conv(**config_feature_extractor)
# feature_extraction_model = transformer_models.Wav2Vec_aggregate(conv_wav2net)

vit = transformer_models.Vanilla_transformer(**config_vanilla_transformer)
# factorized_vit = transformer_models.Factorized_transformer(**config_factorized_transformer)


model = transformer_models.Super_model(feature_extractor=feature_extraction_model,
                                       transformer=vit, 
                                       window_size = config['WINDOW_SIZE'])


loss_func = train_utils.make_mse_loss()
train_step = train_utils.train_step

optimizer = optim.Adam(model.parameters(), 
                       lr=config['lr'], 
                       weight_decay=config['weight_decay'])


print(summary(model, torch.zeros(4, config['n_channels'], 
                                 len(config['freqs']),
                                 config['WINDOW_SIZE']), show_input=False))


-----------------------------------------------------------------------------
            Layer (type)        Output Shape         Param #     Tr. Param #
   Vanilla_transformer-1             [4, 21]          32,005          32,005
Total params: 32,005
Trainable params: 32,005
Non-trainable params: 0
-----------------------------------------------------------------------------


# Model training

In [ ]:
n_runs = 3

for i in range(n_runs):
    

    feature_extraction_model = transformer_models.get_strided_func(**config_feature_extractor)
    # conv_wav2net = transformer_models.wav2vec_conv(**config_feature_extractor)
    # feature_extraction_model = transformer_models.Wav2Vec_aggregate(conv_wav2net)

    vit = transformer_models.Vanilla_transformer(**config_vanilla_transformer)
    # factorized_vit = transformer_models.Factorized_transformer(**config_factorized_transformer)


    model = transformer_models.Super_model(feature_extractor=feature_extraction_model,
                                              transformer=vit, 
                                          window_size = config['WINDOW_SIZE'])


    loss_func = train_utils.make_mse_loss()
    train_step = train_utils.train_step

    optimizer = optim.Adam(model.parameters(), 
                           lr=config['lr'], 
                           weight_decay=config['weight_decay'])
    
    
    parameters = {
        'EPOCHS': 250,
        'model': model, 
        'train_loader': train_loader, 
        'val_loader': val_loader, 
        'loss_function': loss_func,
        'train_step': train_step,
        'optimizer': optimizer, 
        'device': 'cuda', 
        'raw_test_data': test_dataset_prep,
        'show_info': 1, 
        'num_losses': 5,
        'labels': labels_roi,
        'inference_function': inference.model_inference_function, 
        'to_many': config['to_many']
    }



    path_to_save_wandb = 'common/koval_alvi/Checkpoints/wandb_brain'
    
    
    with wandb.init(project="eeg_fmri", config=config, save_code=True):
        
        wandb.define_metric("val/corr_mean", summary="max")

        if i == 0: 
            exp_name = wandb.run.name
        
        wandb.run.name = exp_name +'_run_' + str(i)
        
        print(config)
        print(parameters['model'])
        print(summary(model, torch.zeros(4, config['n_channels'], 
                                         len(config['freqs']),
                                         config['WINDOW_SIZE']), show_input=False))
        model = train_utils.wanb_train_regression(**parameters)
        

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: koval_alvi (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.10 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'n_electrodes': 30, 'sequence_length': 32, 'embed_dim': 16, 'n_roi': 21, 'num_heads': 8, 'mlp_ratio': 4, 'num_layers': 4, 'attn_dropout': 0.1, 'mlp_dropout': 0.5, 'stride': 128, 'dataset_name': 'CWL', 'fps': 250, 'new_fps': 250, 'n_channels': 30, 'test_sec': 60, 'freqs': array([  2.        ,   8.53333333,  15.06666667,  21.6       ,
        28.13333333,  34.66666667,  41.2       ,  47.73333333,
        54.26666667,  60.8       ,  67.33333333,  73.86666667,
        80.4       ,  86.93333333,  93.46666667, 100.        ]), 'bold_delay': 0, 'to_many': False, 'random_subsample': True, 'sample_per_epoch': 2048, 'WINDOW_SIZE': 4096, 'optimizer': 'adam', 'lr': 0.001, 'weight_decay': 1e-05, 'batch_size': 16, 'loss_function': 'mse', 'model_type': 'strided medium vanilla transformer'}
Super_model(
  (transformer): Vanilla_transformer(
    (transformer): TransformerEncoder(
      (layers): ModuleList(
        (0): TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (ou

# 